# Data Gathering and Pre-processing

Import the required libraries...

In [13]:
import csv
import pickle
import os
import pandas as pd
import numpy as np
import nfl_data_py as nfl

## Gathering

NFL play by play data found [here](https://github.com/nflverse/nflverse-data/releases/tag/pbp). In this cell we will convert the csv files to pkl files, so they're easier to work with. This part may take a while, as the csv files are pretty big.

In [20]:
all_seasons = []
# loop through the play by play csv files
for i in range(1999, 2023):
    # open the csv file for the year (i is the year)
    with open(f'./pbp_csv/play_by_play_{i}.csv', 'r') as f:
        reader = csv.reader(f)

        all_games = []
        game = []
        # loop through the rows of the csv
        for row in reader:
            # skip the first row
            if row[0] == 'play_id':
                continue
            # skip the rows that are not plays
            if row[27] == 'GAME':
                continue
            # add the row to the game's dataframe, until the 'desc' column contains END GAME
            # desc is in column AB, which is index 27
            if row[27] == 'END GAME':
                # add game to the list of dataframes and clear the game dataframe
                all_games.append(game)
                game = []
                continue
            else:
                # add the row to the game list
                game.append(row)
        
        all_seasons.append(all_games)

# print the last season to a txt file to check
with open('2022_list.txt', 'w') as f:
    for game in all_seasons[-1]:
        for play in game:
            f.write(str(play) + '\n')
        f.write('\n')

# loop through every game and make them dataframes
for season in all_seasons:
    for game in season:
        game = pd.DataFrame(game)
        # drop the columns that are not needed
        # NOT SURE IF THIS WORKS BECAUSE WE SKIPPED THE FIRST ROW
        # game.drop(columns=['game_id', 'desc'])

# save the last season to a txt file to check
with open('2022_df.txt', 'w') as f:
    for game in all_seasons[-1]:
        for play in game:
            f.write(str(play) + '\n')
        f.write('\n')

Next, gather data on team ratings by season from 